In [2]:
#2020-ml-p1-re
!pip uninstall --y kaggle 
!pip install --upgrade pip
!pip install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v


Uninstalling kaggle-1.5.9:
  Successfully uninstalled kaggle-1.5.9
     |████████████████████████████████| 1.5MB 9.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=5ae56106ebf56cd87a42a78a04bd17890a359646fd7c4a75c0aa8cc9cbef9766
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6


In [3]:
!kaggle competitions download -c 2020ml1re
!unzip 2020ml1re.zip

  0% 0.00/53.2k [00:00<?, ?B/s]
100% 53.2k/53.2k [00:00<00:00, 44.5MB/s]
Archive:  2020ml1re.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


In [11]:
import seaborn as sns
import pandas as pd 
import numpy as np 

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(df_train.shape)
print(df_test.shape)

pd.options.display.max_columns = None
df_train.drop(['Unnamed: 32'], axis=1, inplace=True)
df_test.drop(['Unnamed: 32'], axis=1, inplace=True)

print(df_train.head()) 
print(df_test.head()) 


(398, 32)
(171, 31)
   diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0          0        8.734         16.84           55.27      234.3   
1          0       11.360         17.57           72.49      399.8   
2          1       14.950         17.57           96.85      678.1   
3          1       17.350         23.06          111.00      933.1   
4          0       11.060         17.12           71.25      366.5   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.10390           0.07428         0.00000              0.00000   
1          0.08858           0.05313         0.02783              0.02100   
2          0.11670           0.13050         0.15390              0.08624   
3          0.08662           0.06290         0.02891              0.02837   
4          0.11940           0.10710         0.04063              0.04268   

   symmetry_mean  fractal_dimension_mean  radius_se  texture_se  perimeter_se  \
0         0.198

In [12]:
X = df_train.iloc[:,1:]
y = df_train.iloc[:,[0]]
test_x = df_test.iloc[:,0:]

print(X.shape)
print(y.shape)
print(test_x.shape)


(398, 30)
(398, 1)
(171, 30)


In [115]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=777,stratify=y)

In [116]:
print('y_train class distribution')
print(y_train.value_counts(normalize=True))

print('y_test class distribution')
print(y_test.value_counts(normalize=True))


y_train class distribution
diagnosis
0            0.634228
1            0.365772
dtype: float64
y_test class distribution
diagnosis
0            0.63
1            0.37
dtype: float64


In [130]:
lda = LinearDiscriminantAnalysis()

print(X_train.shape, X_test.shape)
lda.fit(X_train, y_train)
y_pred = lda.predict(test_x)
y_pred

(298, 30) (100, 30)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1])

In [126]:
submit = pd.read_csv('sampleSubmission.csv')

submit

,id,diagnosis
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
166,166,1
167,167,1
168,168,1
169,169,1


In [131]:
for i in range(len(y_pred)):
    submit['diagnosis'][i] = y_pred[i].item()

submit

,id,diagnosis
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
166,166,0
167,167,0
168,168,0
169,169,0


In [132]:
submit.to_csv('result.csv',mode='w',index=False)
!kaggle competitions submit -c 2020ml1re -f result.csv -m "14010974_이기택"

100% 929/929 [00:00<00:00, 2.18kB/s]
Successfully submitted to 2020.ML.중간고사.문제1.다시풀기